In [24]:
from pyspark.sql import SparkSession 
from pyspark.ml.feature import Word2Vec,Tokenizer,StopWordsRemover
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import UserDefinedFunction
from pyspark.sql.types import StringType

spark = SparkSession.builder.appName("SparkML Depression Analysis").getOrCreate()
dataDF= spark.read.option("inferSchema","True")\
                        .option("delimiter","\t")\
                        .csv("datasets/movie_turkish_train.txt")

dataDF = dataDF.withColumnRenamed("_c0","tweet")
dataDF = dataDF.withColumnRenamed("_c1","label")


def removeCharacters(text):
    text = text.replace(","," ")
    text = text.replace("."," ")
    text = text.replace("!"," ")
    text = text.replace("?"," ")
    text = text.replace("cok","çok")
    return text
    
removeUDF = UserDefinedFunction(removeCharacters,StringType())
dataDF = dataDF.withColumn("text",removeUDF('tweet'))

tokenizer = Tokenizer(inputCol='text',outputCol='tokens')
dataDF = tokenizer.transform(dataDF)


remover = StopWordsRemover(inputCol='tokens',outputCol='no-stop-words',stopWords=StopWordsRemover.loadDefaultStopWords('turkish'))
dataDF = remover.transform(dataDF)
dataDF.show()

vSize = 400



w2v = Word2Vec(inputCol='no-stop-words',outputCol='features',vectorSize=vSize,windowSize=23,minCount=5)
dataDF = w2v.fit(dataDF).transform(dataDF)

dataDF.show()


dataDF = dataDF.select('features','label')
dataDF.printSchema()

trainDF,testDF = dataDF.randomSplit([0.75,0.25],seed=123)


mlpClassifier = MultilayerPerceptronClassifier(layers=[vSize,2,2])
model = mlpClassifier.fit(trainDF)


resultDF = model.transform(testDF) #Prediction
#resultDF.show(100)

eva = MulticlassClassificationEvaluator(metricName='accuracy')
accuracy = eva.evaluate(resultDF)
print("Test Accuracy : ",accuracy)

+--------------------+-----+--------------------+--------------------+--------------------+
|               tweet|label|                text|              tokens|       no-stop-words|
+--------------------+-----+--------------------+--------------------+--------------------+
|bunu da gördük uz...|    0|bunu da gördük uz...|[bunu, da, gördük...|[bunu, gördük, uz...|
|filmi begenmeyenl...|    0|filmi begenmeyenl...|[filmi, begenmeye...|[filmi, begenmeye...|
|günde 1 film izle...|    0|günde 1 film izle...|[günde, 1, film, ...|[günde, 1, film, ...|
|afisine bakip ald...|    0|afisine bakip ald...|[afisine, bakip, ...|[afisine, bakip, ...|
|sadece insanlarin...|    0|sadece insanlarin...|[sadece, insanlar...|[sadece, insanlar...|
|ucuz bir aksiyon ...|    0|ucuz bir aksiyon ...|[ucuz, bir, aksiy...|[ucuz, bir, aksiy...|
|olmamis diyor pua...|    0|olmamis diyor pua...|[olmamis, diyor, ...|[olmamis, diyor, ...|
|kesinlikle çok kö...|    0|kesinlikle çok kö...|[kesinlikle, çok,...|[kesinlikl

In [25]:
from pyspark.ml.feature import NGram

wordDataFrame = spark.createDataFrame([
    (0, ["Hi", "I", "heard", "about", "Spark"]),
    (1, ["I", "wish", "Java", "could", "use", "case", "classes"]),
    (2, ["Logistic", "regression", "models", "are", "neat"])
], ["id", "words"])

ngram = NGram(n=2, inputCol="words", outputCol="ngrams")

ngramDataFrame = ngram.transform(wordDataFrame)
ngramDataFrame.select("ngrams").show(truncate=False)

+------------------------------------------------------------------+
|ngrams                                                            |
+------------------------------------------------------------------+
|[Hi I, I heard, heard about, about Spark]                         |
|[I wish, wish Java, Java could, could use, use case, case classes]|
|[Logistic regression, regression models, models are, are neat]    |
+------------------------------------------------------------------+

